In [ ]:
# wr model and predictions

In [1]:
import pandas as pd

In [2]:
stats = pd.read_csv("ffb_stats.csv")

In [3]:
feats = ['Player','Tm','Pos','Rushing Att','Rushing Yds', 'Rushing TD','Year','Pts','Tgt','Rec','Receiving Yds','Receiving TD']

In [4]:
ffb_wr = stats[stats['Pos'] == 'WR']

In [5]:
wr = ffb_wr[feats]

In [6]:
predictors = ['Rushing Att','Rushing Yds', 'Rushing TD','Tgt','Rec','Receiving Yds','Receiving TD']

In [8]:
train = wr[wr['Year'] < 2022]
test = wr[wr['Year'] == 2022]

In [10]:
from sklearn.linear_model import Ridge

wrm = Ridge(alpha=.1)

In [11]:
wrm.fit(train[predictors], train["Pts"])

Ridge(alpha=0.1)

In [12]:
predictions = wrm.predict(test[predictors])

In [13]:
predictions = pd.DataFrame(predictions, columns=["Pred Pts"], index=test.index)

In [14]:
comb = pd.concat([test[["Player", "Pts"]], predictions], axis=1)

In [15]:
comb

,Player,Pts,Pred Pts
235,Justin Jefferson,237.0,231.637593
249,Davante Adams,230.0,232.762936
293,Tyreek Hill,218.0,216.557036
317,A.J. Brown,211.0,212.842864
343,Stefon Diggs,203.0,204.045275
...,...,...,...
7458,C.J. Board,-2.0,-0.833171
7465,Mason Kinsey,-2.0,-1.139087
7545,Keke Coutee,-6.0,0.594435
7563,KaVontae Turpin,-11.0,0.601302


In [16]:
from sklearn.metrics import mean_squared_error

In [17]:
mean_squared_error(comb["Pts"], comb["Pred Pts"])

10.324083153073735

In [18]:
import pickle

In [19]:
with open('wrm.pkl', 'wb') as model_file:
    pickle.dump(wrm, model_file)

In [20]:
wr_proj = pd.read_csv('wr_projections.csv')

In [22]:
del wr_proj ['Proj Pts']

In [23]:
wr_proj.columns

Index(['Player', 'Team', 'Ru Att', 'Ru Yds', 'Ru TDs', 'Tgt', 'Rec', 'Rec Yds',
       'Rec TDs'],
      dtype='object')

In [24]:
wr_proj.rename(columns={'Ru Att': 'Rushing Att'}, inplace=True)

In [25]:
wr_proj.rename(columns={'Ru Yds': 'Rushing Yds'}, inplace=True)

In [26]:
wr_proj.rename(columns={'Ru TDs': 'Rushing TD'}, inplace=True)

In [27]:
wr_proj.rename(columns={'Rec Yds': 'Receiving Yds'}, inplace=True)

In [28]:
wr_proj.rename(columns={'Rec TDs': 'Receiving TD'}, inplace=True)

In [29]:
wr_proj.rename(columns={'Team': 'Tm'}, inplace=True)

In [30]:
wr_proj = wr_proj.fillna(0)

In [31]:
wr_proj

,Player,Tm,Rushing Att,Rushing Yds,Rushing TD,Tgt,Rec,Receiving Yds,Receiving TD
0,Justin Jefferson,MIN,2.0,7.5,0.1,159.15,110.20,1569.8,8.10
1,Ja'Marr Chase,CIN,2.0,4.5,0.0,150.15,98.95,1335.5,10.60
2,Cooper Kupp,LA,2.0,9.5,0.1,152.35,111.80,1347.5,8.35
3,Tyreek Hill,MIA,2.0,9.5,0.1,145.40,98.50,1348.5,7.60
4,Stefon Diggs,BUF,0.0,0.0,0.0,145.25,99.65,1243.0,8.95
...,...,...,...,...,...,...,...,...,...
181,Trent Taylor,CIN,0.0,0.0,0.0,5.90,3.10,36.4,0.10
182,James Proche,BAL,0.0,0.0,0.0,4.80,3.10,34.1,0.10
183,Derius Davis,LAC,0.0,0.0,0.0,2.90,2.40,29.1,0.10
184,Grant DuBose,GB,0.0,0.0,0.0,3.95,2.45,27.4,0.10


In [32]:
wr_pred = wrm.predict(wr_proj[predictors])

In [33]:
wr_pred = pd.DataFrame(wr_pred, columns=["Pred Pts"], index=wr_proj.index)

In [34]:
wr_comb = pd.concat([wr_proj[["Player", "Tm"]], wr_pred], axis=1)

In [35]:
wr_comb = wr_comb.sort_values("Pred Pts", ascending=False)
wr_comb["Pred Rk"] = list(range(1,wr_comb.shape[0]+1))

In [37]:
wr_comb.head(10)

,Player,Tm,Pred Pts,Pred Rk
0,Justin Jefferson,MIN,201.940051,1
1,Ja'Marr Chase,CIN,193.366918,2
2,Cooper Kupp,LA,180.612423,3
3,Tyreek Hill,MIA,177.302478,4
4,Stefon Diggs,BUF,173.373485,5
5,Davante Adams,LV,169.885508,6
7,A.J. Brown,PHI,167.768946,7
6,CeeDee Lamb,DAL,163.737599,8
9,Garrett Wilson,NYJ,154.189223,9
10,Jaylen Waddle,MIA,152.375120,10


In [38]:
wr_comb.to_csv('proj_wr.csv')